In [1]:
import ccxt
import pandas as pd
from IPython.display import display, HTML

# Function to fetch ETH/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    return df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create a heatmap-style pivot table
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean and median for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)

    # Add the average and median as the last row
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value

    # Apply color gradients for heatmap (Green for positive, Red for negative)
    def color_map(val):
        color = 'white'  # Default color for no value
        if isinstance(val, (int, float)):  # Check if value is numeric
            if val > 0:
                color = f'rgb({255 - int(val*2.5)}, 255, {255 - int(val*2.5)})'  # Green shades for positive
            elif val < 0:
                color = f'rgb(255, {255 + int(val*2.5)}, {255 - int(abs(val)*2.5)})'  # Red shades for negative
        return f'background-color: {color};'

    # Apply heatmap coloring
    styled_table = pivot_table.style \
        .applymap(color_map)  # Apply the heatmap color function to each cell

    # Additional table styling
    styled_table = styled_table.set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ADA/USDT'
    eth_data = fetch_data(symbol)

    # Calculate indicators
    eth_data = calculate_max_drawdown(eth_data)
    eth_data = calculate_high_to_open_change(eth_data)
    eth_data = calculate_weekly_return(eth_data)

    # Create and display the pivot tables with heatmap style
    styled_table_max_drawdown = create_pivot_table(eth_data, 'max_drawdown')
    display_table("ETH/USDT Weekly Max Drawdown Heatmap (Month-Letter-WeekOfMonth)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(eth_data, 'high_to_open_change')
    display_table("ETH/USDT Weekly Open to High % Change Heatmap (Month-Letter-WeekOfMonth)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(eth_data, 'weekly_return')
    display_table("ETH/USDT Weekly Returns Heatmap (Open to Close, Month-Letter-WeekOfMonth)", styled_table_weekly_return)

# Run the main function
main()


C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\681103241.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-6.136746,-9.446023,-12.085184,-32.362207,-16.887943,-26.181719,-11.162621,nan,-25.834069,-16.407241,-25.780520,-13.190676,nan,-3.508030,-15.757492,-1.363221,-8.425341,-24.480767,-17.908046,-24.677188,-14.302635,-1.074576,nan,-29.777481,-19.797155,-11.376646,-17.362270,nan,-8.118905,-18.251624,-2.289224,-6.047656,-7.950530,-4.256145,-24.512860,-46.832398,-11.247390,nan,-35.277247,-11.107534,-0.344828,-18.997482,-6.992023
2019,-19.535462,-0.126775,-6.433566,-10.469667,nan,-6.171548,-7.036347,-1.514423,-0.408457,-7.195441,-2.061856,-1.725176,-8.603025,nan,-1.374570,-15.674051,-16.030897,-8.989224,-6.881001,-10.495229,-1.053981,-14.244220,-6.035698,nan,-20.143213,-1.507856,-7.421918,-16.090412,nan,-8.010210,-26.316449,-15.608600,-11.966650,-6.112140,-15.597148,-18.707793,-9.279368,-12.871287,nan,-2.238806,-5.024694,-2.207931,-28.495503,-3.928956,-1.830369,-9.643809,-10.973137,-4.976303,nan,-1.462479,-4.925884,-24.216841,-9.922791,nan,-8.543984,-7.825413,-16.840373,-8.289703,-5.685815
2020,-0.635654,-3.115100,-1.547619,-1.763787,nan,-3.176597,-8.387201,-10.252238,-26.648217,-7.172626,-58.867397,-19.910847,-1.627219,-0.249288,-0.031338,-7.653214,-5.043674,-3.896387,nan,-13.212171,-5.737705,-2.047582,-0.526418,nan,-4.207820,-15.655690,-9.340659,-4.924097,0.000000,-0.947529,-6.824809,-7.547778,-15.269078,nan,-1.352664,-14.994487,-14.853767,-15.931171,-27.269630,-7.155191,-7.971396,-15.566249,-11.498223,nan,-7.744734,-4.093128,-6.323927,-16.212406,nan,-10.522031,-7.529103,-0.945744,-23.045296,-15.204608,-18.588976,-3.252402,-22.017782,-1.209026,nan
2021,-6.894545,-24.122836,-26.464795,-13.981780,nan,-3.626973,-4.413922,-18.659833,-26.333500,-21.256256,-12.888134,-4.979998,-12.450131,-3.263491,-1.966922,-15.170602,-27.930751,-1.072411,nan,-5.351624,-15.071906,-56.090775,-5.017130,-3.187099,-19.134220,-16.083017,-29.876595,-4.140818,nan,-12.015053,-14.742306,-13.644828,-1.196776,nan,-4.047166,-2.265713,-13.693635,-8.687616,-5.192982,-31.295088,-12.441860,-16.333771,-8.431550,nan,-6.219458,-5.390589,-3.058387,-14.865503,nan,-2.748092,-4.356436,-16.617647,-22.833787,-25.517241,-13.497823,-12.118959,-3.456592,-11.958763,nan
2022,-18.432511,-8.632479,-35.010631,-15.391459,-2.801932,-9.894921,-11.888782,-19.871106,-4.550758,-5.474453,-1.397713,-1.820250,-6.756757,nan,-14.080944,-11.111111,-4.376368,-16.937853,nan,-8.107423,-45.718531,-17.803410,-17.762186,-0.975306,-13.779736,-14.233279,-5.980615,-11.165544,nan,-2.521930,-13.016216,-0.468959,-11.914062,nan,-5.328425,-4.852161,-23.669886,-7.996542,-1.327743,-9.910626,-13.632797,-3.663167,-6.409682,nan,-0.882002,-17.494090,-10.883068,-2.430268,-6.334730,-22.904750,-6.032131,-5.352564,-4.030710,nan,-5.307263,-17.019199,-7.330827,-7.644788,nan
2023,-1.320528,-0.672269,-7.725200,-7.837407,-8.148893,-9.915881,-5.245812,-12.068102,-12.953930,-11.691395,-4.275318,-4.553364,-5.396290,nan,-2.830189,-1.181608,-16.023936,-3.241574,nan,-5.610311,-7.313830,-3.881402,-1.860078,-5.747126,-41.690962,-7.134976,-3.371648,-10.993151,nan,-5.315501,-2.180795,-4.737679,-5.422446,-8.520748,-3.559206,-17.789292,-7.510174,-5.426945,nan,-5.314576,-4.977921,-2.225820,-0.988468,nan,-5.039489,-5.538222,-3.233630,-0.909091,-3.925550,-2.456859,-11.342894,-7.526042,-4.672173,nan,-0.937183,-14.141584,-6.373816,-5.372529,nan
2024,-17.881798,-6.107179,-10.245667,-11.011727,-1.366510,-4.175958,-2.0129

C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\681103241.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,20.151853,37.286932,7.169457,0.407526,0.349953,1.733448,23.330555,nan,1.632833,4.519774,6.867327,11.501720,nan,14.180051,0.773374,38.437215,6.612741,0.159764,4.352490,2.358321,1.658277,18.547174,nan,2.272944,3.622234,34.373211,2.359488,nan,2.220656,5.020673,18.606924,0.815851,6.781734,6.856404,1.325019,0.176423,17.092902,nan,0.717017,2.994205,54.448276,12.130922,14.500235
2019,12.235587,16.556795,2.587413,0.807241,nan,12.683054,0.163094,21.466346,6.511293,12.087390,13.950428,29.669007,19.336477,nan,42.840779,2.137371,1.378491,8.839176,5.106197,18.945934,43.569292,0.964331,14.549084,nan,2.044417,22.402433,12.950267,10.300341,nan,4.521697,2.720662,10.191298,3.874448,5.371275,3.743316,0.219018,3.948667,2.687412,nan,7.417458,3.371269,19.121115,1.083401,8.261572,12.013406,1.889993,10.035479,6.445498,nan,10.357228,4.720639,2.732726,20.446097,nan,0.075834,0.232438,0.246846,1.657941,3.370457
2020,11.961861,21.436114,11.523810,31.056039,nan,12.688442,16.855204,3.710334,0.602312,14.696205,2.540200,26.151560,16.235207,19.373219,15.794422,10.612855,30.064807,18.393557,nan,9.781499,9.857083,12.324493,68.648859,nan,22.297389,0.462161,10.596546,11.276949,27.211562,41.385634,7.179447,24.699621,1.710783,nan,13.818100,0.971610,4.336551,4.618805,9.112587,7.716382,3.102324,14.961952,5.319779,nan,15.148699,7.285017,5.047965,1.127820,nan,15.874631,2.469020,52.832255,25.402590,4.740190,0.195276,14.437808,3.198321,35.514321,nan
2021,72.266667,31.790144,4.325446,6.806994,nan,105.687094,48.240028,42.369021,35.328089,2.682074,7.894900,39.963512,8.598106,4.885572,13.666089,23.318510,3.750734,29.395050,nan,38.162358,39.848262,1.252065,39.474686,19.973335,3.833999,3.508321,0.764269,11.779655,nan,0.006842,1.861327,5.696631,13.905398,nan,13.662990,57.828283,25.591914,9.866913,8.807018,0.549639,0.503876,7.709155,5.258386,nan,5.508663,2.786661,6.487488,3.916942,nan,9.109415,17.722772,2.696078,0.217984,10.344828,7.329463,0.892193,19.855305,9.553265,nan
2022,0.000000,21.538462,16.087881,2.935943,13.816425,10.595447,6.903164,6.659506,17.969662,4.987835,16.391360,35.381115,5.236486,nan,4.974705,0.682261,6.783370,2.135593,nan,14.745376,1.863857,2.741558,3.305022,43.037975,17.991189,11.582382,8.475974,3.857930,nan,8.114035,0.756757,22.532381,8.593750,nan,4.340244,12.736922,2.440737,4.171169,18.798043,4.111221,1.941938,7.773062,3.832362,nan,4.338498,0.898345,1.485282,21.513394,7.912250,3.481721,4.546832,3.461538,4.446577,nan,2.110490,3.709730,1.278195,2.895753,nan
2023,19.647859,24.268908,10.376283,6.030818,4.024145,2.854958,15.709970,2.603906,0.325203,0.504451,11.491813,12.238979,15.036537,nan,6.132075,18.597483,0.066489,7.872395,nan,0.555977,0.957447,2.614555,6.774014,0.914316,0.477074,6.509746,16.858238,2.636986,nan,3.429355,33.415406,5.945946,0.567465,1.045296,3.559206,1.105354,0.813910,6.337761,nan,2.071122,2.448816,4.411170,10.337727,nan,1.391501,0.858034,6.871463,13.106061,18.443316,15.560105,6.779661,4.010417,4.026846,nan,63.981763,14.343366,12.196382,14.360534,nan
2024,7.257114,24.853387,4.399162,0.636056,9.463594,14.615852,18.024007,3.551824,29.173009,9.697802,13.105373,2.582918,5.688669,nan,0.215351,5.922281,9.693987,4.625551,3.411560,3.887312,12.083143,9.289384,3.689151,nan,9.742441,1.598379,1.031917,6.261510,nan,7.898089,28.756850,5.860551,0.760626,4.911591,3.602557,5.525031,20.107720,0.805404,nan,2.990937,7.669617,6.704076,18.

C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\681103241.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,10.210951,28.742898,-4.405396,-19.887858,-7.799986,-24.925653,18.027920,nan,-23.451015,-7.506053,-18.408425,8.605273,nan,2.007608,-1.760807,18.867262,-3.949932,-19.970505,-14.536398,-9.047704,-8.143322,12.819686,nan,-27.409034,-7.955743,28.591872,-5.275459,nan,-0.552227,-15.298287,7.774986,-4.636105,5.055722,-0.388098,-19.017927,-38.540497,9.264092,nan,-25.669216,-6.632325,50.655172,-2.426184,13.139371
2019,-18.208212,8.696755,-4.731935,-6.457926,nan,12.186192,-3.145387,-0.096154,1.057184,8.216576,9.300285,23.390171,13.660758,nan,28.579611,-6.311923,-12.893642,-6.179239,-3.942392,6.345600,22.674833,-1.080516,13.081259,nan,-18.150685,16.434364,5.560997,-15.099597,nan,-3.123860,-25.739719,3.385813,-2.958967,-5.623842,-2.459893,-7.629129,-2.290227,-10.587998,nan,5.314337,0.150311,4.866024,-24.059689,4.440258,6.393400,-4.385752,6.842372,-1.232227,nan,5.106689,2.645382,-22.283937,13.125536,nan,-2.123357,-5.759298,-5.759737,-0.756254,1.406800
2020,9.563710,10.876452,6.595238,24.358116,nan,11.055276,-0.678733,0.000000,-25.752889,-5.878482,-37.240736,0.408618,3.772189,13.603989,4.888750,6.098655,29.416737,6.617327,nan,-2.899735,7.797394,-0.058502,44.082667,nan,17.128941,-11.704217,2.577185,2.411022,22.265138,29.240958,-2.261802,19.740344,-9.914461,nan,8.452283,-4.341235,-12.541421,-3.285032,-21.137796,2.600907,-6.025870,13.361683,-4.401895,nan,9.954564,0.779196,-0.912732,-7.640977,nan,8.425766,-5.689072,45.335988,14.184883,-4.758190,-2.954331,5.173981,-4.501111,33.283765,nan
2021,46.196364,26.288386,-7.174935,-2.464211,nan,91.820450,27.613487,30.379176,19.324269,-13.891788,-6.924730,12.656664,0.355935,-0.653034,6.894801,1.042128,-14.511770,21.604033,nan,33.134846,30.234402,-42.878880,19.916254,6.475779,-7.185022,-8.353145,-6.184266,9.238359,nan,-7.731782,-12.398962,3.978331,6.993406,nan,8.444275,51.985129,24.839618,5.397412,2.105263,-11.336311,-11.511628,-3.328953,2.039891,nan,-2.754331,-1.416172,-1.760890,-7.314771,nan,2.849873,0.990099,-10.049020,-13.133515,-13.605016,-2.322206,-7.583643,16.961415,-5.360825,nan
2022,-15.094340,20.512821,-20.340184,-7.918149,10.434783,-8.756567,-10.738255,-8.055854,-3.967328,-4.379562,11.689962,34.698521,0.168919,nan,-13.406408,-10.916179,-3.063457,-10.790960,nan,-6.219914,-19.219341,-9.461718,-11.004431,17.742270,-13.585903,-1.141925,1.031140,-6.899367,nan,1.403509,-3.178378,14.336757,0.800781,nan,2.228250,7.941622,-18.770852,-7.218500,17.283951,1.251241,-12.181248,-0.290373,-6.006275,nan,0.834327,-12.458629,-2.214421,12.013256,-0.887355,-17.980602,-5.425887,0.192308,3.039028,nan,-4.593420,-13.472177,-2.593985,-3.513514,nan
2023,19.007603,17.882353,7.297605,5.605739,-1.332998,-7.188376,9.722604,-7.586380,-8.672087,-2.136499,4.548211,3.161253,7.223159,nan,2.044025,15.926021,-13.874113,1.775148,nan,-4.978519,-1.303191,-2.911051,6.274292,-1.436782,-27.908826,-4.008827,11.915709,-0.136986,nan,-2.469136,10.657756,0.826709,-0.504414,-7.412100,-0.958248,-6.632124,-2.552719,-2.884250,nan,-2.696366,-0.802890,-1.780656,9.514003,nan,-3.610380,-3.549142,6.669361,11.893939,15.702200,12.138052,0.130378,0.859375,1.935983,nan,50.658561,-2.404574,1.963824,0.354790,nan
2024,-16.736824,6.188069,-4.170634,-2.544226,0.632266,9.750659,14.367498,-4.520504,23.101640,-1.565934,-4.898814,-5.077781,0.494667,nan,-9.367790,-20.702528,6.847849,-7.829395,-0.478053,-4.411444,6.715395,-1.947774,-2.532198,

In [2]:
import ccxt
import pandas as pd
from IPython.display import display, HTML

# Function to fetch ADA/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    # Map months to letters (A = Jan, B = Feb, ..., L = Dec)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    # Create unique "week number" format: MonthLetter-MonthNumber-WeekOfMonth
    

    
    # Create unique "week number" format: MonthLetter-WeekOfMonth
    # df['unique_week_number'] = df['month_letter'] + '-W' + df['week_of_month'].astype(str)
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    return df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create and style the pivot table with positive percentages in green
# Function to create and style the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)
    max_value = pivot_table.max(axis=0)
    min_value = pivot_table.min(axis=0)

    # Add the average, median, max, and min as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value
    pivot_table.loc['Max'] = max_value
    pivot_table.loc['Min'] = min_value

    # Function to color positive values green and negative values red
    def color_positive_negative(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'background-color: lightgreen; color: white;'  # Green for positive values
            elif val < 0:
                return 'background-color: pink; color: white;'  # Red for negative values
        return ''  # No change for non-numeric values

    # Style the table
    styled_table = pivot_table.style \
        .format('{:.2f}%') \
        .applymap(color_positive_negative)  # Apply the coloring function to the table
    
    # Set additional styles
    styled_table.set_table_styles([ 
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%'), ('display', 'block'), ('overflow-x', 'auto'), ('position', 'relative')]},  # Set relative position for the table container
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
        
        # Add a line below the "Avg" row to separate it from the rest of the table
        {'selector': 'tr:nth-last-child(5)', 'props': [('border-bottom', '2px solid black')]},  # Row for "Avg"
        
        # # Freeze the first column (Year) and the header row
        # {'selector': 'thead', 'props': [('position', 'sticky'), ('top', '0'), ('z-index', '1')]},  # Freeze header row
        # {'selector': 'thead th:first-child', 'props': [('position', 'sticky'), ('left', '0'), ('background-color', '#f2f2f2'), ('z-index', '2')]},  # Freeze the first column in header
        # {'selector': 'tbody td:first-child', 'props': [('position', 'sticky'), ('left', '0'), ('background-color', '#f2f2f2'), ('z-index', '2')]},  # Freeze the first column in body
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table



# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ADA/USDT'
    ADA_data = fetch_data(symbol)

    # Calculate indicators
    ADA_data = calculate_max_drawdown(ADA_data)
    ADA_data = calculate_high_to_open_change(ADA_data)
    ADA_data = calculate_weekly_return(ADA_data)

    # Create and display the pivot tables
    styled_table_max_drawdown = create_pivot_table(ADA_data, 'max_drawdown')
    display_table("ADA/USDT Weekly Max Drawdown (Month-Letter-WeekOfMonth)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(ADA_data, 'high_to_open_change')
    display_table("ADA/USDT Weekly Open to High % Change (Month-Letter-WeekOfMonth)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(ADA_data, 'weekly_return')
    display_table("ADA/USDT Weekly Returns (Open to Close, Month-Letter-WeekOfMonth)", styled_table_weekly_return)

# Run the main function
main()


C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\4025824737.py:82: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-6.14%,-9.45%,-12.09%,-32.36%,-16.89%,-26.18%,-11.16%,nan%,-25.83%,-16.41%,-25.78%,-13.19%,nan%,-3.51%,-15.76%,-1.36%,-8.43%,-24.48%,-17.91%,-24.68%,-14.30%,-1.07%,nan%,-29.78%,-19.80%,-11.38%,-17.36%,nan%,-8.12%,-18.25%,-2.29%,-6.05%,-7.95%,-4.26%,-24.51%,-46.83%,-11.25%,nan%,-35.28%,-11.11%,-0.34%,-19.00%,-6.99%
2019,-19.54%,-0.13%,-6.43%,-10.47%,nan%,-6.17%,-7.04%,-1.51%,-0.41%,-7.20%,-2.06%,-1.73%,-8.60%,nan%,-1.37%,-15.67%,-16.03%,-8.99%,-6.88%,-10.50%,-1.05%,-14.24%,-6.04%,nan%,-20.14%,-1.51%,-7.42%,-16.09%,nan%,-8.01%,-26.32%,-15.61%,-11.97%,-6.11%,-15.60%,-18.71%,-9.28%,-12.87%,nan%,-2.24%,-5.02%,-2.21%,-28.50%,-3.93%,-1.83%,-9.64%,-10.97%,-4.98%,nan%,-1.46%,-4.93%,-24.22%,-9.92%,nan%,-8.54%,-7.83%,-16.84%,-8.29%,-5.69%
2020,-0.64%,-3.12%,-1.55%,-1.76%,nan%,-3.18%,-8.39%,-10.25%,-26.65%,-7.17%,-58.87%,-19.91%,-1.63%,-0.25%,-0.03%,-7.65%,-5.04%,-3.90%,nan%,-13.21%,-5.74%,-2.05%,-0.53%,nan%,-4.21%,-15.66%,-9.34%,-4.92%,0.00%,-0.95%,-6.82%,-7.55%,-15.27%,nan%,-1.35%,-14.99%,-14.85%,-15.93%,-27.27%,-7.16%,-7.97%,-15.57%,-11.50%,nan%,-7.74%,-4.09%,-6.32%,-16.21%,nan%,-10.52%,-7.53%,-0.95%,-23.05%,-15.20%,-18.59%,-3.25%,-22.02%,-1.21%,nan%
2021,-6.89%,-24.12%,-26.46%,-13.98%,nan%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,nan%,-5.35%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-4.14%,nan%,-12.02%,-14.74%,-13.64%,-1.20%,nan%,-4.05%,-2.27%,-13.69%,-8.69%,-5.19%,-31.30%,-12.44%,-16.33%,-8.43%,nan%,-6.22%,-5.39%,-3.06%,-14.87%,nan%,-2.75%,-4.36%,-16.62%,-22.83%,-25.52%,-13.50%,-12.12%,-3.46%,-11.96%,nan%
2022,-18.43%,-8.63%,-35.01%,-15.39%,-2.80%,-9.89%,-11.89%,-19.87%,-4.55%,-5.47%,-1.40%,-1.82%,-6.76%,nan%,-14.08%,-11.11%,-4.38%,-16.94%,nan%,-8.11%,-45.72%,-17.80%,-17.76%,-0.98%,-13.78%,-14.23%,-5.98%,-11.17%,nan%,-2.52%,-13.02%,-0.47%,-11.91%,nan%,-5.33%,-4.85%,-23.67%,-8.00%,-1.33%,-9.91%,-13.63%,-3.66%,-6.41%,nan%,-0.88%,-17.49%,-10.88%,-2.43%,-6.33%,-22.90%,-6.03%,-5.35%,-4.03%,nan%,-5.31%,-17.02%,-7.33%,-7.64%,nan%
2023,-1.32%,-0.67%,-7.73%,-7.84%,-8.15%,-9.92%,-5.25%,-12.07%,-12.95%,-11.69%,-4.28%,-4.55%,-5.40%,nan%,-2.83%,-1.18%,-16.02%,-3.24%,nan%,-5.61%,-7.31%,-3.88%,-1.86%,-5.75%,-41.69%,-7.13%,-3.37%,-10.99%,nan%,-5.32%,-2.18%,-4.74%,-5.42%,-8.52%,-3.56%,-17.79%,-7.51%,-5.43%,nan%,-5.31%,-4.98%,-2.23%,-0.99%,nan%,-5.04%,-5.54%,-3.23%,-0.91%,-3.93%,-2.46%,-11.34%,-7.53%,-4.67%,nan%,-0.94%,-14.14%,-6.37%,-5.37%,nan%
2024,-17.88%,-6.11%,-10.25%,-11.01%,-1.37%,-4.18%,-2.01%,-8.30%,-3.06%,-21.57%,-12.24%,-16.51%,-2.60%,nan%,-14.04%,-32.22%,-9.54%,-11.09%,-9.08%,-5.42%,-2.90%,-4.97%,-3.82%,nan%,-4.75%,-9.95%,-14.21%,-5.02%,nan%,-19.36%,-4.79%,-3.98%,-14.63%,-18.10%,-19.93%,-2.26%,-1.97%,-15.12%,nan%,-8.34%,-2.65%,-3.60%,-1.79%,-16.55%,-7.25%,-2.64%,-13.92%,-3.62%,nan%,-4.16%,-12.04%,-0.34%,-14.52%,nan%,-8.09%,-23.76%,nan%,nan%,nan%
Avg,-10.78%,-7.13%,-14.57%,-10.08%,-4.11%,-6.16%,-6.50%,-11.78%,-12.33%,-12.39%,-15.29%,-8.25%,-6.24%,-1.76%,-5.72%,-13.84%,-12.16%,-7.81%,-9.35%,-11.51%,-13.53%,-17.89%,-6.60%,-3.30%,-18.51%,-11.57%,-13.71%,-9.36%,0.00%,-7.38%,-11.95%,-6.77%,-9.83%,-14.30%,-9.67%,-12.22%,-12.18%,-9.59%,-11.26%,-13.43%,-9.50%,-7.85%,-10.71%,-10.24%,-5.30%,-9.01%,-7.24%,-7.01%,-6.07%,-6.93%,-10.11%,-14.55%,-12.90%,-20.36%,-12.89%,-12.75%,-9.39%,-8.91%,-6.34%
Median,-12.39%,-4.61%,-8.99%,-10.74%,-2.80%,-5.1

C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\4025824737.py:82: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,20.15%,37.29%,7.17%,0.41%,0.35%,1.73%,23.33%,nan%,1.63%,4.52%,6.87%,11.50%,nan%,14.18%,0.77%,38.44%,6.61%,0.16%,4.35%,2.36%,1.66%,18.55%,nan%,2.27%,3.62%,34.37%,2.36%,nan%,2.22%,5.02%,18.61%,0.82%,6.78%,6.86%,1.33%,0.18%,17.09%,nan%,0.72%,2.99%,54.45%,12.13%,14.50%
2019,12.24%,16.56%,2.59%,0.81%,nan%,12.68%,0.16%,21.47%,6.51%,12.09%,13.95%,29.67%,19.34%,nan%,42.84%,2.14%,1.38%,8.84%,5.11%,18.95%,43.57%,0.96%,14.55%,nan%,2.04%,22.40%,12.95%,10.30%,nan%,4.52%,2.72%,10.19%,3.87%,5.37%,3.74%,0.22%,3.95%,2.69%,nan%,7.42%,3.37%,19.12%,1.08%,8.26%,12.01%,1.89%,10.04%,6.45%,nan%,10.36%,4.72%,2.73%,20.45%,nan%,0.08%,0.23%,0.25%,1.66%,3.37%
2020,11.96%,21.44%,11.52%,31.06%,nan%,12.69%,16.86%,3.71%,0.60%,14.70%,2.54%,26.15%,16.24%,19.37%,15.79%,10.61%,30.06%,18.39%,nan%,9.78%,9.86%,12.32%,68.65%,nan%,22.30%,0.46%,10.60%,11.28%,27.21%,41.39%,7.18%,24.70%,1.71%,nan%,13.82%,0.97%,4.34%,4.62%,9.11%,7.72%,3.10%,14.96%,5.32%,nan%,15.15%,7.29%,5.05%,1.13%,nan%,15.87%,2.47%,52.83%,25.40%,4.74%,0.20%,14.44%,3.20%,35.51%,nan%
2021,72.27%,31.79%,4.33%,6.81%,nan%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,nan%,38.16%,39.85%,1.25%,39.47%,19.97%,3.83%,3.51%,0.76%,11.78%,nan%,0.01%,1.86%,5.70%,13.91%,nan%,13.66%,57.83%,25.59%,9.87%,8.81%,0.55%,0.50%,7.71%,5.26%,nan%,5.51%,2.79%,6.49%,3.92%,nan%,9.11%,17.72%,2.70%,0.22%,10.34%,7.33%,0.89%,19.86%,9.55%,nan%
2022,0.00%,21.54%,16.09%,2.94%,13.82%,10.60%,6.90%,6.66%,17.97%,4.99%,16.39%,35.38%,5.24%,nan%,4.97%,0.68%,6.78%,2.14%,nan%,14.75%,1.86%,2.74%,3.31%,43.04%,17.99%,11.58%,8.48%,3.86%,nan%,8.11%,0.76%,22.53%,8.59%,nan%,4.34%,12.74%,2.44%,4.17%,18.80%,4.11%,1.94%,7.77%,3.83%,nan%,4.34%,0.90%,1.49%,21.51%,7.91%,3.48%,4.55%,3.46%,4.45%,nan%,2.11%,3.71%,1.28%,2.90%,nan%
2023,19.65%,24.27%,10.38%,6.03%,4.02%,2.85%,15.71%,2.60%,0.33%,0.50%,11.49%,12.24%,15.04%,nan%,6.13%,18.60%,0.07%,7.87%,nan%,0.56%,0.96%,2.61%,6.77%,0.91%,0.48%,6.51%,16.86%,2.64%,nan%,3.43%,33.42%,5.95%,0.57%,1.05%,3.56%,1.11%,0.81%,6.34%,nan%,2.07%,2.45%,4.41%,10.34%,nan%,1.39%,0.86%,6.87%,13.11%,18.44%,15.56%,6.78%,4.01%,4.03%,nan%,63.98%,14.34%,12.20%,14.36%,nan%
2024,7.26%,24.85%,4.40%,0.64%,9.46%,14.62%,18.02%,3.55%,29.17%,9.70%,13.11%,2.58%,5.69%,nan%,0.22%,5.92%,9.69%,4.63%,3.41%,3.89%,12.08%,9.29%,3.69%,nan%,9.74%,1.60%,1.03%,6.26%,nan%,7.90%,28.76%,5.86%,0.76%,4.91%,3.60%,5.53%,20.11%,0.81%,nan%,2.99%,7.67%,6.70%,18.26%,0.03%,3.11%,6.48%,2.19%,7.12%,nan%,97.57%,38.79%,64.28%,13.74%,nan%,15.37%,0.00%,nan%,nan%,nan%
Avg,20.56%,23.41%,8.22%,8.05%,9.10%,26.52%,17.65%,13.39%,14.98%,7.44%,10.90%,24.33%,11.69%,12.13%,13.94%,10.21%,10.27%,15.51%,5.23%,12.36%,15.50%,4.42%,22.82%,21.31%,8.29%,7.23%,8.22%,8.23%,27.21%,11.36%,10.78%,16.19%,5.15%,2.87%,6.73%,11.53%,8.41%,6.72%,12.24%,3.88%,3.24%,13.58%,6.64%,4.14%,6.25%,3.60%,7.25%,7.72%,11.05%,22.69%,10.91%,18.60%,12.20%,7.54%,12.83%,5.23%,15.20%,12.69%,8.94%
Median,12.10%,22.90%,7.39%,4.48%,9.46%,12.69%,16.28%,5.18%,12.24%,7.34%,12.30%,27.91%,11.82%,12.13%,9.90%,8.27%,6.78%,8.84%,5.11%,9.78%,9.86%,2.61%,14.55%,19.97%,3.83%,4.52%,8.48%,10.30%,27.21%,7.90%,2.72%,10.19%,3.87%,2.98%,4.34%,2.36%,3.95%,4.62%,9.11%,2.99%,3.10%,7.77%,5.26%,4.14%,4.34%,2.79%,6.49%,6.45%,7.91%,10.36%,4.72%,3.46%,13.74%,7.54%,2.11%,2.99%,7.70%,10.84%,8.94%


C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\4025824737.py:82: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,10.21%,28.74%,-4.41%,-19.89%,-7.80%,-24.93%,18.03%,nan%,-23.45%,-7.51%,-18.41%,8.61%,nan%,2.01%,-1.76%,18.87%,-3.95%,-19.97%,-14.54%,-9.05%,-8.14%,12.82%,nan%,-27.41%,-7.96%,28.59%,-5.28%,nan%,-0.55%,-15.30%,7.77%,-4.64%,5.06%,-0.39%,-19.02%,-38.54%,9.26%,nan%,-25.67%,-6.63%,50.66%,-2.43%,13.14%
2019,-18.21%,8.70%,-4.73%,-6.46%,nan%,12.19%,-3.15%,-0.10%,1.06%,8.22%,9.30%,23.39%,13.66%,nan%,28.58%,-6.31%,-12.89%,-6.18%,-3.94%,6.35%,22.67%,-1.08%,13.08%,nan%,-18.15%,16.43%,5.56%,-15.10%,nan%,-3.12%,-25.74%,3.39%,-2.96%,-5.62%,-2.46%,-7.63%,-2.29%,-10.59%,nan%,5.31%,0.15%,4.87%,-24.06%,4.44%,6.39%,-4.39%,6.84%,-1.23%,nan%,5.11%,2.65%,-22.28%,13.13%,nan%,-2.12%,-5.76%,-5.76%,-0.76%,1.41%
2020,9.56%,10.88%,6.60%,24.36%,nan%,11.06%,-0.68%,0.00%,-25.75%,-5.88%,-37.24%,0.41%,3.77%,13.60%,4.89%,6.10%,29.42%,6.62%,nan%,-2.90%,7.80%,-0.06%,44.08%,nan%,17.13%,-11.70%,2.58%,2.41%,22.27%,29.24%,-2.26%,19.74%,-9.91%,nan%,8.45%,-4.34%,-12.54%,-3.29%,-21.14%,2.60%,-6.03%,13.36%,-4.40%,nan%,9.95%,0.78%,-0.91%,-7.64%,nan%,8.43%,-5.69%,45.34%,14.18%,-4.76%,-2.95%,5.17%,-4.50%,33.28%,nan%
2021,46.20%,26.29%,-7.17%,-2.46%,nan%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,nan%,33.13%,30.23%,-42.88%,19.92%,6.48%,-7.19%,-8.35%,-6.18%,9.24%,nan%,-7.73%,-12.40%,3.98%,6.99%,nan%,8.44%,51.99%,24.84%,5.40%,2.11%,-11.34%,-11.51%,-3.33%,2.04%,nan%,-2.75%,-1.42%,-1.76%,-7.31%,nan%,2.85%,0.99%,-10.05%,-13.13%,-13.61%,-2.32%,-7.58%,16.96%,-5.36%,nan%
2022,-15.09%,20.51%,-20.34%,-7.92%,10.43%,-8.76%,-10.74%,-8.06%,-3.97%,-4.38%,11.69%,34.70%,0.17%,nan%,-13.41%,-10.92%,-3.06%,-10.79%,nan%,-6.22%,-19.22%,-9.46%,-11.00%,17.74%,-13.59%,-1.14%,1.03%,-6.90%,nan%,1.40%,-3.18%,14.34%,0.80%,nan%,2.23%,7.94%,-18.77%,-7.22%,17.28%,1.25%,-12.18%,-0.29%,-6.01%,nan%,0.83%,-12.46%,-2.21%,12.01%,-0.89%,-17.98%,-5.43%,0.19%,3.04%,nan%,-4.59%,-13.47%,-2.59%,-3.51%,nan%
2023,19.01%,17.88%,7.30%,5.61%,-1.33%,-7.19%,9.72%,-7.59%,-8.67%,-2.14%,4.55%,3.16%,7.22%,nan%,2.04%,15.93%,-13.87%,1.78%,nan%,-4.98%,-1.30%,-2.91%,6.27%,-1.44%,-27.91%,-4.01%,11.92%,-0.14%,nan%,-2.47%,10.66%,0.83%,-0.50%,-7.41%,-0.96%,-6.63%,-2.55%,-2.88%,nan%,-2.70%,-0.80%,-1.78%,9.51%,nan%,-3.61%,-3.55%,6.67%,11.89%,15.70%,12.14%,0.13%,0.86%,1.94%,nan%,50.66%,-2.40%,1.96%,0.35%,nan%
2024,-16.74%,6.19%,-4.17%,-2.54%,0.63%,9.75%,14.37%,-4.52%,23.10%,-1.57%,-4.90%,-5.08%,0.49%,nan%,-9.37%,-20.70%,6.85%,-7.83%,-0.48%,-4.41%,6.72%,-1.95%,-2.53%,nan%,-0.52%,-6.17%,-8.81%,3.24%,nan%,-11.69%,24.52%,3.52%,-8.88%,-15.47%,-4.82%,2.01%,15.17%,-13.98%,nan%,2.42%,-0.15%,3.87%,13.05%,-10.09%,-2.46%,4.71%,-6.85%,-1.68%,nan%,76.68%,19.02%,45.55%,12.40%,nan%,3.82%,-7.81%,nan%,nan%,nan%
Avg,4.12%,15.07%,-3.75%,1.76%,3.24%,18.14%,6.19%,1.69%,0.85%,-3.27%,-3.92%,11.54%,4.28%,6.48%,3.27%,-2.48%,0.30%,4.85%,-2.94%,0.15%,5.59%,-11.89%,12.55%,7.59%,-10.52%,-3.21%,-1.76%,0.19%,22.27%,1.09%,-1.45%,9.24%,-2.63%,-12.12%,-0.52%,4.90%,-0.61%,-2.82%,-0.58%,-4.27%,-5.50%,6.47%,-2.16%,-2.82%,1.11%,-4.52%,1.36%,0.20%,6.62%,12.40%,-1.05%,3.01%,5.83%,-9.18%,2.40%,-5.50%,9.45%,3.60%,7.27%
Median,-2.77%,14.38%,-4.45%,-2.50%,0.63%,10.40%,4.52%,-2.31%,-1.46%,-3.26%,-0.18%,7.91%,2.13%,6.48%,3.47%,-2.63%,-3.06%,1.78%,-3.94%,-4.41%,6.72%,-2.91%,13.08%,6.48%,-13.59%,-6.17%,1.03%,2.41%,22.27%,-2.47%,-2.26%,3.98%,-2.96%,-11.44%,-0.96%,-4.34%,-2.55%,-3.29%,2.11%,1.25%,-6.0

In [3]:
import ccxt
import pandas as pd
from IPython.display import display, HTML
from datetime import timedelta

# Function to fetch ADA/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    # Create unique "week number" format: MonthLetter-MonthNumber-WeekOfMonth
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    return df

# Function to filter data for the 18 months after each halving event
def filter_halving_data(df):
    halving_dates = ['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20']
    halving_dates = pd.to_datetime(halving_dates)

    # Filter the DataFrame to only include data from 18 months after each halving date
    filtered_df = pd.DataFrame()
    for halving_date in halving_dates:
        start_date = halving_date
        end_date = start_date + timedelta(weeks=18*4)  # 18 months = 18 * 4 weeks
        filtered_df = pd.concat([filtered_df, df[(df.index >= start_date) & (df.index <= end_date)]])

    return filtered_df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create and style the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)
    max_value = pivot_table.max(axis=0)
    min_value = pivot_table.min(axis=0)

    # Add the average, median, max, and min as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value
    pivot_table.loc['Max'] = max_value
    pivot_table.loc['Min'] = min_value

    # Function to color positive values green and negative values red
    def color_positive_negative(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'background-color: lightgreen; color: white;'  # Green for positive values
            elif val < 0:
                return 'background-color: pink; color: white;'  # Red for negative values
        return ''  # No change for non-numeric values

    # Style the table
    styled_table = pivot_table.style \
        .format('{:.2f}%') \
        .applymap(color_positive_negative)  # Apply the coloring function to the table
    
    # Set additional styles
    styled_table.set_table_styles([ 
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%'), ('display', 'block'), ('overflow-x', 'auto'), ('position', 'relative')]},  # Set relative position for the table container
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
        
        # Add a line below the "Avg" row to separate it from the rest of the table
        {'selector': 'tr:nth-last-child(5)', 'props': [('border-bottom', '2px solid black')]},  # Row for "Avg"
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ADA/USDT'
    ADA_data = fetch_data(symbol)

    # Filter data for 18 months after each halving
    ADA_data = filter_halving_data(ADA_data)

    # Calculate indicators
    ADA_data = calculate_max_drawdown(ADA_data)
    ADA_data = calculate_high_to_open_change(ADA_data)
    ADA_data = calculate_weekly_return(ADA_data)

    # Create and display the pivot tables
    styled_table_max_drawdown = create_pivot_table(ADA_data, 'max_drawdown')
    display_table("ADA/USDT Weekly Max Drawdown (Month-Letter-WeekOfMonth)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(ADA_data, 'high_to_open_change')
    display_table("ADA/USDT Weekly Open to High % Change (Month-Letter-WeekOfMonth)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(ADA_data, 'weekly_return')
    display_table("ADA/USDT Weekly Returns (Open to Close, Month-Letter-WeekOfMonth)", styled_table_weekly_return)

# Run the main function
main()


# 第一次减半	2012-11-28	50→25
# 第二次减半	2016-07-10	25→12.5
# 第三次减半	2020-05-12	12.5→6.25
# 第四次减半	2024-04-20	6.25→3.125




# 然後help me build a new col in the df fetched, and name it "減半後的第幾周“（make it english name）

#  find該周是減半後的第幾周然後填入col of the df
# 再create一個col“第幾次減半”

# 然後幫我改變三幅table的code，我要“減半後的第幾周”作為x axis ，第幾次減半作為y axis

# 三個table：
# 應該有大约（18x4）個col （18个月）
# 和4個row（四個減半）



C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\2769260656.py:90: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-2.05%,-0.53%,nan%,-4.21%,-15.66%,-9.34%,-4.92%,0.00%,-0.95%,-6.82%,-7.55%,-15.27%,nan%,-1.35%,-14.99%,-14.85%,-15.93%,-27.27%,-7.16%,-7.97%,-15.57%,-11.50%,nan%,-7.74%,-4.09%,-6.32%,-16.21%,-10.52%,-7.53%,-0.95%,-23.05%,-15.20%,-18.59%,-3.25%,-22.02%,-1.21%
2021,-6.89%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,nan%,-5.35%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-4.14%,nan%,-12.02%,-14.74%,-13.64%,-1.20%,nan%,-4.05%,-2.27%,-13.69%,-8.69%,-5.19%,-31.30%,-12.44%,-16.33%,-8.43%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-11.09%,-9.08%,-5.42%,-2.90%,-4.97%,-3.82%,nan%,-4.75%,-9.95%,-14.21%,-5.02%,nan%,-19.36%,-4.79%,-3.98%,-14.63%,-18.10%,-19.93%,-2.26%,-1.97%,-15.12%,nan%,-8.34%,-2.65%,-3.60%,-1.79%,-16.55%,-7.25%,-2.64%,-13.92%,-3.62%,-4.16%,-12.04%,-0.34%,-14.52%,nan%,-8.09%,-23.76%,nan%,nan%
Avg,-6.89%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-6.08%,-9.08%,-5.38%,-8.99%,-21.03%,-3.12%,-3.19%,-9.36%,-13.90%,-17.81%,-4.70%,0.00%,-10.78%,-8.79%,-8.39%,-10.37%,-18.10%,-8.44%,-6.51%,-10.17%,-13.25%,-16.23%,-15.60%,-7.69%,-11.83%,-7.24%,-16.55%,-7.50%,-3.37%,-10.12%,-9.92%,-7.34%,-9.79%,-0.64%,-18.78%,-15.20%,-13.34%,-13.51%,-22.02%,-1.21%
Median,-6.89%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-6.08%,-9.08%,-5.38%,-8.99%,-4.97%,-3.82%,-3.19%,-4.75%,-15.66%,-14.21%,-4.92%,0.00%,-12.02%,-6.82%,-7.55%,-14.63%,-18.10%,-4.05%,-2.27%,-13.69%,-15.12%,-16.23%,-8.34%,-7.97%,-15.57%,-8.43%,-16.55%,-7.50%,-3.37%,-10.12%,-9.92%,-7.34%,-9.79%,-0.64%,-18.78%,-15.20%,-13.34%,-13.51%,-22.02%,-1.21%
Max,-6.89%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,-9.08%,-5.35%,-2.90%,-2.05%,-0.53%,-3.19%,-4.21%,-9.95%,-9.34%,-4.14%,0.00%,-0.95%,-4.79%,-3.98%,-1.20%,-18.10%,-1.35%,-2.26%,-1.97%,-8.69%,-5.19%,-7.16%,-2.65%,-3.60%,-1.79%,-16.55%,-7.25%,-2.64%,-6.32%,-3.62%,-4.16%,-7.53%,-0.34%,-14.52%,-15.20%,-8.09%,-3.25%,-22.02%,-1.21%
Min,-6.89%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-11.09%,-9.08%,-5.42%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-5.02%,0.00%,-19.36%,-14.74%,-13.64%,-15.27%,-18.10%,-19.93%,-14.99%,-14.85%,-15.93%,-27.27%,-31.30%,-12.44%,-16.33%,-11.50%,-16.55%,-7.74%,-4.09%,-13.92%,-16.21%,-10.52%,-12.04%,-0.95%,-23.05%,-15.20%,-18.59%,-23.76%,-22.02%,-1.21%


C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\2769260656.py:90: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,12.32%,68.65%,nan%,22.30%,0.46%,10.60%,11.28%,27.21%,41.39%,7.18%,24.70%,1.71%,nan%,13.82%,0.97%,4.34%,4.62%,9.11%,7.72%,3.10%,14.96%,5.32%,nan%,15.15%,7.29%,5.05%,1.13%,15.87%,2.47%,52.83%,25.40%,4.74%,0.20%,14.44%,3.20%,35.51%
2021,72.27%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,nan%,38.16%,39.85%,1.25%,39.47%,19.97%,3.83%,3.51%,0.76%,11.78%,nan%,0.01%,1.86%,5.70%,13.91%,nan%,13.66%,57.83%,25.59%,9.87%,8.81%,0.55%,0.50%,7.71%,5.26%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,4.63%,3.41%,3.89%,12.08%,9.29%,3.69%,nan%,9.74%,1.60%,1.03%,6.26%,nan%,7.90%,28.76%,5.86%,0.76%,4.91%,3.60%,5.53%,20.11%,0.81%,nan%,2.99%,7.67%,6.70%,18.26%,0.03%,3.11%,6.48%,2.19%,7.12%,97.57%,38.79%,64.28%,13.74%,nan%,15.37%,0.00%,nan%,nan%
Avg,72.27%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,17.01%,3.41%,21.02%,25.97%,7.62%,37.27%,19.97%,11.96%,1.86%,4.13%,9.77%,27.21%,16.43%,12.60%,12.09%,5.46%,4.91%,10.36%,21.44%,16.68%,5.10%,8.96%,3.75%,3.76%,9.79%,9.61%,0.03%,9.13%,6.88%,3.62%,4.12%,56.72%,20.63%,58.55%,19.57%,4.74%,7.78%,7.22%,3.20%,35.51%
Median,72.27%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,17.01%,3.41%,21.02%,25.97%,9.29%,39.47%,19.97%,9.74%,1.60%,1.03%,11.28%,27.21%,7.90%,7.18%,5.86%,1.71%,4.91%,13.66%,5.53%,20.11%,4.62%,8.96%,2.99%,3.10%,7.71%,5.32%,0.03%,9.13%,6.88%,3.62%,4.12%,56.72%,20.63%,58.55%,19.57%,4.74%,7.78%,7.22%,3.20%,35.51%
Max,72.27%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,3.41%,38.16%,39.85%,12.32%,68.65%,19.97%,22.30%,3.51%,10.60%,11.78%,27.21%,41.39%,28.76%,24.70%,13.91%,4.91%,13.82%,57.83%,25.59%,9.87%,9.11%,7.72%,7.67%,14.96%,18.26%,0.03%,15.15%,7.29%,5.05%,7.12%,97.57%,38.79%,64.28%,25.40%,4.74%,15.37%,14.44%,3.20%,35.51%
Min,72.27%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,4.63%,3.41%,3.89%,12.08%,1.25%,3.69%,19.97%,3.83%,0.46%,0.76%,6.26%,27.21%,0.01%,1.86%,5.70%,0.76%,4.91%,3.60%,0.97%,4.34%,0.81%,8.81%,0.55%,0.50%,6.70%,5.26%,0.03%,3.11%,6.48%,2.19%,1.13%,15.87%,2.47%,52.83%,13.74%,4.74%,0.20%,0.00%,3.20%,35.51%


C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\2769260656.py:90: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-0.06%,44.08%,nan%,17.13%,-11.70%,2.58%,2.41%,22.27%,29.24%,-2.26%,19.74%,-9.91%,nan%,8.45%,-4.34%,-12.54%,-3.29%,-21.14%,2.60%,-6.03%,13.36%,-4.40%,nan%,9.95%,0.78%,-0.91%,-7.64%,8.43%,-5.69%,45.34%,14.18%,-4.76%,-2.95%,5.17%,-4.50%,33.28%
2021,46.20%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,nan%,33.13%,30.23%,-42.88%,19.92%,6.48%,-7.19%,-8.35%,-6.18%,9.24%,nan%,-7.73%,-12.40%,3.98%,6.99%,nan%,8.44%,51.99%,24.84%,5.40%,2.11%,-11.34%,-11.51%,-3.33%,2.04%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,-7.83%,-0.48%,-4.41%,6.72%,-1.95%,-2.53%,nan%,-0.52%,-6.17%,-8.81%,3.24%,nan%,-11.69%,24.52%,3.52%,-8.88%,-15.47%,-4.82%,2.01%,15.17%,-13.98%,nan%,2.42%,-0.15%,3.87%,13.05%,-10.09%,-2.46%,4.71%,-6.85%,-1.68%,76.68%,19.02%,45.55%,12.40%,nan%,3.82%,-7.78%,nan%,nan%
Avg,46.20%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,6.89%,-0.48%,14.36%,18.47%,-14.96%,20.49%,6.48%,3.14%,-8.74%,-4.14%,4.96%,22.27%,3.27%,3.29%,9.08%,-3.93%,-15.47%,4.02%,16.55%,9.16%,-3.96%,-9.52%,-2.11%,-5.89%,4.63%,3.56%,-10.09%,3.75%,2.74%,-3.88%,-4.66%,42.55%,6.67%,45.44%,13.29%,-4.76%,0.43%,-1.30%,-4.50%,33.28%
Median,46.20%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,6.89%,-0.48%,14.36%,18.47%,-1.95%,19.92%,6.48%,-0.52%,-8.35%,-6.18%,3.24%,22.27%,-7.73%,-2.26%,3.98%,-8.88%,-15.47%,8.44%,2.01%,15.17%,-3.29%,-9.52%,2.42%,-6.03%,3.87%,2.04%,-10.09%,3.75%,2.74%,-3.88%,-4.66%,42.55%,6.67%,45.44%,13.29%,-4.76%,0.43%,-1.30%,-4.50%,33.28%
Max,46.20%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,-0.48%,33.13%,30.23%,-0.06%,44.08%,6.48%,17.13%,-6.17%,2.58%,9.24%,22.27%,29.24%,24.52%,19.74%,6.99%,-15.47%,8.45%,51.99%,24.84%,5.40%,2.11%,2.60%,-0.15%,13.36%,13.05%,-10.09%,9.95%,4.71%,-0.91%,-1.68%,76.68%,19.02%,45.55%,14.18%,-4.76%,3.82%,5.17%,-4.50%,33.28%
Min,46.20%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,-7.83%,-0.48%,-4.41%,6.72%,-42.88%,-2.53%,6.48%,-7.19%,-11.70%,-8.81%,2.41%,22.27%,-11.69%,-12.40%,3.52%,-9.91%,-15.47%,-4.82%,-4.34%,-12.54%,-13.98%,-21.14%,-11.34%,-11.51%,-3.33%,-4.40%,-10.09%,-2.46%,0.78%,-6.85%,-7.64%,8.43%,-5.69%,45.34%,12.40%,-4.76%,-2.95%,-7.78%,-4.50%,33.28%


In [4]:
import ccxt
import pandas as pd
from IPython.display import display, HTML
from datetime import timedelta

# Function to fetch ADA/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    # Create unique "week number" format: MonthLetter-MonthNumber-WeekOfMonth
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)
    
    return df

# Function to filter data for the 18 months after each halving event
def filter_halving_data(df):
    halving_dates = ['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20']
    halving_dates = pd.to_datetime(halving_dates)

    # Filter the DataFrame to only include data from 18 months after each halving date
    filtered_df = pd.DataFrame()
    for halving_date in halving_dates:
        start_date = halving_date
        end_date = start_date + timedelta(weeks=18*4)  # 18 months = 18 * 4 weeks
        filtered_df = pd.concat([filtered_df, df[(df.index >= start_date) & (df.index <= end_date)]])

    return filtered_df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to add the halving week number and halving count columns
def add_halving_columns(df):
    halving_dates = pd.to_datetime(['2012-11-28', '2016-07-10', '2020-05-12', '2024-04-20'])
    halving_counts = [1, 2, 3, 4]
    
    # Create new columns to store the week after halving and halving count
    df['week_after_halving'] = None
    df['halving_count'] = None

    # Iterate over each halving event
    for i, halving_date in enumerate(halving_dates):
        # Calculate the weeks since each halving event
        df.loc[df.index >= halving_date, 'week_after_halving'] = ((df.index[df.index >= halving_date] - halving_date).days // 7) + 1
        df.loc[df.index >= halving_date, 'halving_count'] = halving_counts[i]
    
    return df

# Function to create and style the pivot table with positive percentages in green
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('week_after_halving')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='halving_count', columns='week_after_halving', aggfunc='mean')

    # Calculate mean, median, max, and min for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)
    max_value = pivot_table.max(axis=0)
    min_value = pivot_table.min(axis=0)

    # Add the average, median, max, and min as the last rows
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value
    pivot_table.loc['Max'] = max_value
    pivot_table.loc['Min'] = min_value

    # Function to color positive values green and negative values red
    def color_positive_negative(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'background-color: lightgreen; color: white;'  # Green for positive values
            elif val < 0:
                return 'background-color: pink; color: white;'  # Red for negative values
        return ''  # No change for non-numeric values

    # Style the table
    styled_table = pivot_table.style \
        .format('{:.2f}%') \
        .applymap(color_positive_negative)  # Apply the coloring function to the table
    
    # Set additional styles
    styled_table.set_table_styles([ 
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%'), ('display', 'block'), ('overflow-x', 'auto'), ('position', 'relative')]},  # Set relative position for the table container
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
        
        # Add a line below the "Avg" row to separate it from the rest of the table
        {'selector': 'tr:nth-last-child(5)', 'props': [('border-bottom', '2px solid black')]},  # Row for "Avg"
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'ADA/USDT'
    ADA_data = fetch_data(symbol)

    # Filter data for 18 months after each halving
    ADA_data = filter_halving_data(ADA_data)

    # Add halving week and halving count columns
    ADA_data = add_halving_columns(ADA_data)

    # Calculate indicators
    ADA_data = calculate_max_drawdown(ADA_data)
    ADA_data = calculate_high_to_open_change(ADA_data)
    ADA_data = calculate_weekly_return(ADA_data)

    # Create and display the pivot tables
    styled_table_max_drawdown = create_pivot_table(ADA_data, 'max_drawdown')
    display_table("ADA/USDT Weekly Max Drawdown (Week After Halving)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(ADA_data, 'high_to_open_change')
    display_table("ADA/USDT Weekly Open to High % Change (Week After Halving)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(ADA_data, 'weekly_return')
    display_table("ADA/USDT Weekly Returns (Open to Close, Week After Halving)", styled_table_weekly_return)

# Run the main function
main()



C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\4016401147.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,-2.05%,-0.53%,-4.21%,-15.66%,-9.34%,-4.92%,0.00%,-0.95%,-6.82%,-7.55%,-15.27%,-1.35%,-14.99%,-14.85%,-15.93%,-27.27%,-7.16%,-7.97%,-15.57%,-11.50%,-7.74%,-4.09%,-6.32%,-16.21%,-10.52%,-7.53%,-0.95%,-23.05%,-15.20%,-18.59%,-3.25%,-22.02%,-1.21%,-6.89%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,-5.35%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-4.14%,-12.02%,-14.74%,-13.64%,-1.20%,-4.05%,-2.27%,-13.69%,-8.69%,-5.19%,-31.30%,-12.44%,-16.33%,-8.43%
4,-11.09%,-9.08%,-5.42%,-2.90%,-4.97%,-3.82%,-4.75%,-9.95%,-14.21%,-5.02%,-19.36%,-4.79%,-3.98%,-14.63%,-18.10%,-19.93%,-2.26%,-1.97%,-15.12%,-8.34%,-2.65%,-3.60%,-1.79%,-16.55%,-7.25%,-2.64%,-13.92%,-3.62%,-4.16%,-12.04%,-0.34%,-14.52%,-8.09%,-23.76%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,-6.57%,-4.80%,-4.81%,-9.28%,-7.15%,-4.37%,-2.37%,-5.45%,-10.52%,-6.29%,-17.32%,-3.07%,-9.49%,-14.74%,-17.02%,-23.60%,-4.71%,-4.97%,-15.34%,-9.92%,-5.20%,-3.85%,-4.06%,-16.38%,-8.88%,-5.08%,-7.43%,-13.33%,-9.68%,-15.32%,-1.80%,-18.27%,-4.65%,-15.33%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,-5.35%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-4.14%,-12.02%,-14.74%,-13.64%,-1.20%,-4.05%,-2.27%,-13.69%,-8.69%,-5.19%,-31.30%,-12.44%,-16.33%,-8.43%
Median,-6.57%,-4.80%,-4.81%,-9.28%,-7.15%,-4.37%,-2.37%,-5.45%,-10.52%,-6.29%,-17.32%,-3.07%,-9.49%,-14.74%,-17.02%,-23.60%,-4.71%,-4.97%,-15.34%,-9.92%,-5.20%,-3.85%,-4.06%,-16.38%,-8.88%,-5.08%,-7.43%,-13.33%,-9.68%,-15.32%,-1.80%,-18.27%,-4.65%,-15.33%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,-5.35%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-4.14%,-12.02%,-14.74%,-13.64%,-1.20%,-4.05%,-2.27%,-13.69%,-8.69%,-5.19%,-31.30%,-12.44%,-16.33%,-8.43%
Max,-2.05%,-0.53%,-4.21%,-2.90%,-4.97%,-3.82%,0.00%,-0.95%,-6.82%,-5.02%,-15.27%,-1.35%,-3.98%,-14.63%,-15.93%,-19.93%,-2.26%,-1.97%,-15.12%,-8.34%,-2.65%,-3.60%,-1.79%,-16.21%,-7.25%,-2.64%,-0.95%,-3.62%,-4.16%,-12.04%,-0.34%,-14.52%,-1.21%,-6.89%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,-5.35%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-4.14%,-12.02%,-14.74%,-13.64%,-1.20%,-4.05%,-2.27%,-13.69%,-8.69%,-5.19%,-31.30%,-12.44%,-16.33%,-8.43%
Min,-11.09%,-9.08%,-5.42%,-15.66%,-9.34%,-4.92%,-4.75%,-9.95%,-14.21%,-7.55%,-19.36%,-4.79%,-14.99%,-14.85%,-18.10%,-27.27%,-7.16%,-7.97%,-15.57%,-11.50%,-7.74%,-4.09%,-6.32%,-16.55%,-10.52%,-7.53%,-13.92%,-23.05%,-15.20%,-18.59%,-3.25%,-22.02%,-8.09%,-23.76%,-24.12%,-26.46%,-13.98%,-3.63%,-4.41%,-18.66%,-26.33%,-21.26%,-12.89%,-4.98%,-12.45%,-3.26%,-1.97%,-15.17%,-27.93%,-1.07%,-5.35%,-15.07%,-56.09%,-5.02%,-3.19%,-19.13%,-16.08%,-29.88%,-4.14%,-12.02%,-14.74%,-13.64%,-1.20%,-4.05%,-2.27%,-13.69%,-8.69%,-5.19%,-31.30%,-12.44%,-16.33%,-8.43%


C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\4016401147.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,12.32%,68.65%,22.30%,0.46%,10.60%,11.28%,27.21%,41.39%,7.18%,24.70%,1.71%,13.82%,0.97%,4.34%,4.62%,9.11%,7.72%,3.10%,14.96%,5.32%,15.15%,7.29%,5.05%,1.13%,15.87%,2.47%,52.83%,25.40%,4.74%,0.20%,14.44%,3.20%,35.51%,72.27%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,38.16%,39.85%,1.25%,39.47%,19.97%,3.83%,3.51%,0.76%,11.78%,0.01%,1.86%,5.70%,13.91%,13.66%,57.83%,25.59%,9.87%,8.81%,0.55%,0.50%,7.71%,5.26%
4,4.63%,3.41%,3.89%,12.08%,9.29%,3.69%,9.74%,1.60%,1.03%,6.26%,7.90%,28.76%,5.86%,0.76%,4.91%,3.60%,5.53%,20.11%,0.81%,2.99%,7.67%,6.70%,18.26%,0.03%,3.11%,6.48%,2.19%,7.12%,97.57%,38.79%,64.28%,13.74%,15.37%,0.00%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,8.48%,36.03%,13.09%,6.27%,9.94%,7.48%,18.48%,21.49%,4.11%,15.48%,4.80%,21.29%,3.42%,2.55%,4.77%,6.36%,6.62%,11.61%,7.88%,4.16%,11.41%,6.99%,11.65%,0.58%,9.49%,4.48%,27.51%,16.26%,51.16%,19.49%,39.36%,8.47%,25.44%,36.13%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,38.16%,39.85%,1.25%,39.47%,19.97%,3.83%,3.51%,0.76%,11.78%,0.01%,1.86%,5.70%,13.91%,13.66%,57.83%,25.59%,9.87%,8.81%,0.55%,0.50%,7.71%,5.26%
Median,8.48%,36.03%,13.09%,6.27%,9.94%,7.48%,18.48%,21.49%,4.11%,15.48%,4.80%,21.29%,3.42%,2.55%,4.77%,6.36%,6.62%,11.61%,7.88%,4.16%,11.41%,6.99%,11.65%,0.58%,9.49%,4.48%,27.51%,16.26%,51.16%,19.49%,39.36%,8.47%,25.44%,36.13%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,38.16%,39.85%,1.25%,39.47%,19.97%,3.83%,3.51%,0.76%,11.78%,0.01%,1.86%,5.70%,13.91%,13.66%,57.83%,25.59%,9.87%,8.81%,0.55%,0.50%,7.71%,5.26%
Max,12.32%,68.65%,22.30%,12.08%,10.60%,11.28%,27.21%,41.39%,7.18%,24.70%,7.90%,28.76%,5.86%,4.34%,4.91%,9.11%,7.72%,20.11%,14.96%,5.32%,15.15%,7.29%,18.26%,1.13%,15.87%,6.48%,52.83%,25.40%,97.57%,38.79%,64.28%,13.74%,35.51%,72.27%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,38.16%,39.85%,1.25%,39.47%,19.97%,3.83%,3.51%,0.76%,11.78%,0.01%,1.86%,5.70%,13.91%,13.66%,57.83%,25.59%,9.87%,8.81%,0.55%,0.50%,7.71%,5.26%
Min,4.63%,3.41%,3.89%,0.46%,9.29%,3.69%,9.74%,1.60%,1.03%,6.26%,1.71%,13.82%,0.97%,0.76%,4.62%,3.60%,5.53%,3.10%,0.81%,2.99%,7.67%,6.70%,5.05%,0.03%,3.11%,2.47%,2.19%,7.12%,4.74%,0.20%,14.44%,3.20%,15.37%,0.00%,31.79%,4.33%,6.81%,105.69%,48.24%,42.37%,35.33%,2.68%,7.89%,39.96%,8.60%,4.89%,13.67%,23.32%,3.75%,29.40%,38.16%,39.85%,1.25%,39.47%,19.97%,3.83%,3.51%,0.76%,11.78%,0.01%,1.86%,5.70%,13.91%,13.66%,57.83%,25.59%,9.87%,8.81%,0.55%,0.50%,7.71%,5.26%


C:\Users\nicol\AppData\Local\Temp\ipykernel_27896\4016401147.py:108: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_positive_negative)  # Apply the coloring function to the table


week_after_halving,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
halving_count,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,-0.06%,44.08%,17.13%,-11.70%,2.58%,2.41%,22.27%,29.24%,-2.26%,19.74%,-9.91%,8.45%,-4.34%,-12.54%,-3.29%,-21.14%,2.60%,-6.03%,13.36%,-4.40%,9.95%,0.78%,-0.91%,-7.64%,8.43%,-5.69%,45.34%,14.18%,-4.76%,-2.95%,5.17%,-4.50%,33.28%,46.20%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,33.13%,30.23%,-42.88%,19.92%,6.48%,-7.19%,-8.35%,-6.18%,9.24%,-7.73%,-12.40%,3.98%,6.99%,8.44%,51.99%,24.84%,5.40%,2.11%,-11.34%,-11.51%,-3.33%,2.04%
4,-7.83%,-0.48%,-4.41%,6.72%,-1.95%,-2.53%,-0.52%,-6.17%,-8.81%,3.24%,-11.69%,24.52%,3.52%,-8.88%,-15.47%,-4.82%,2.01%,15.17%,-13.98%,2.42%,-0.15%,3.87%,13.05%,-10.09%,-2.46%,4.71%,-6.85%,-1.68%,76.68%,19.02%,45.55%,12.40%,3.82%,-7.76%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
Avg,-3.94%,21.80%,6.36%,-2.49%,0.31%,-0.06%,10.88%,11.54%,-5.53%,11.49%,-10.80%,16.48%,-0.41%,-10.71%,-9.38%,-12.98%,2.31%,4.57%,-0.31%,-0.99%,4.90%,2.32%,6.07%,-8.86%,2.98%,-0.49%,19.24%,6.25%,35.96%,8.03%,25.36%,3.95%,18.55%,19.22%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,33.13%,30.23%,-42.88%,19.92%,6.48%,-7.19%,-8.35%,-6.18%,9.24%,-7.73%,-12.40%,3.98%,6.99%,8.44%,51.99%,24.84%,5.40%,2.11%,-11.34%,-11.51%,-3.33%,2.04%
Median,-3.94%,21.80%,6.36%,-2.49%,0.31%,-0.06%,10.88%,11.54%,-5.53%,11.49%,-10.80%,16.48%,-0.41%,-10.71%,-9.38%,-12.98%,2.31%,4.57%,-0.31%,-0.99%,4.90%,2.32%,6.07%,-8.86%,2.98%,-0.49%,19.24%,6.25%,35.96%,8.03%,25.36%,3.95%,18.55%,19.22%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,33.13%,30.23%,-42.88%,19.92%,6.48%,-7.19%,-8.35%,-6.18%,9.24%,-7.73%,-12.40%,3.98%,6.99%,8.44%,51.99%,24.84%,5.40%,2.11%,-11.34%,-11.51%,-3.33%,2.04%
Max,-0.06%,44.08%,17.13%,6.72%,2.58%,2.41%,22.27%,29.24%,-2.26%,19.74%,-9.91%,24.52%,3.52%,-8.88%,-3.29%,-4.82%,2.60%,15.17%,13.36%,2.42%,9.95%,3.87%,13.05%,-7.64%,8.43%,4.71%,45.34%,14.18%,76.68%,19.02%,45.55%,12.40%,33.28%,46.20%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,33.13%,30.23%,-42.88%,19.92%,6.48%,-7.19%,-8.35%,-6.18%,9.24%,-7.73%,-12.40%,3.98%,6.99%,8.44%,51.99%,24.84%,5.40%,2.11%,-11.34%,-11.51%,-3.33%,2.04%
Min,-7.83%,-0.48%,-4.41%,-11.70%,-1.95%,-2.53%,-0.52%,-6.17%,-8.81%,3.24%,-11.69%,8.45%,-4.34%,-12.54%,-15.47%,-21.14%,2.01%,-6.03%,-13.98%,-4.40%,-0.15%,0.78%,-0.91%,-10.09%,-2.46%,-5.69%,-6.85%,-1.68%,-4.76%,-2.95%,5.17%,-4.50%,3.82%,-7.76%,26.29%,-7.17%,-2.46%,91.82%,27.61%,30.38%,19.32%,-13.89%,-6.92%,12.66%,0.36%,-0.65%,6.89%,1.04%,-14.51%,21.60%,33.13%,30.23%,-42.88%,19.92%,6.48%,-7.19%,-8.35%,-6.18%,9.24%,-7.73%,-12.40%,3.98%,6.99%,8.44%,51.99%,24.84%,5.40%,2.11%,-11.34%,-11.51%,-3.33%,2.04%
